In [1]:
import pandas as pd 
import pyodbc
import warnings
warnings.simplefilter('ignore')
import sqlite3


Connection with Outdoors

In [2]:

DB = {
    'servername': 'tcp:dwhunitedoutdoors.database.windows.net,1433',
    'database': 'DWH_United_Outdoors',
    'username': 'onderzoeker',
    'password': 'Wachtwoord1.' 
}

conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=' + DB['servername'] + ';'
    'DATABASE=' + DB['database'] + ';'
    'UID=' + DB['username'] + ';'
    'PWD=' + DB['password'] + ';'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=30;'
)

export_conn = pyodbc.connect(conn_str)
export_cursor = export_conn.cursor()
export_cursor


In [3]:
DB_AW = {'servername': 'LAPTOPTK\\SQLEXPRESS',
'database': 'AdventureWorks2019'}

DB_AW_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=' + DB_AW['servername'] +
                              ';DATABASE=' + DB_AW['database'] + ';Trusted_Connection=yes')


Connection with northwind

In [4]:
DB = {'servername': 'LAPTOPTK\SQLEXPRESS',
'database': 'Northwind'}

DB_NW_conn = pyodbc.connect('DRIVER={SQL Server}; SERVER=' + DB['servername'] +
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor

Customer table in dataframe van northwind

In [5]:
query_customers = "SELECT * FROM Customers"

# Execute the query and load data into a Pandas DataFrame
customers_df = pd.read_sql(query_customers, DB_NW_conn)

# Close the database connection

# Now you have the Customers table in a Pandas DataFrame
print(customers_df.head())  # Display the first few rows to verify

  CustomerID                         CompanyName         ContactName  \
0      ALFKI                 Alfreds Futterkiste        Maria Anders   
1      ANATR  Ana Trujillo Emparedados y helados        Ana Trujillo   
2      ANTON             Antonio Moreno Taquería      Antonio Moreno   
3      AROUT                     Around the Horn        Thomas Hardy   
4      BERGS                  Berglunds snabbköp  Christina Berglund   

           ContactTitle                        Address         City Region  \
0  Sales Representative                  Obere Str. 57       Berlin   None   
1                 Owner  Avda. de la Constitución 2222  México D.F.   None   
2                 Owner                Mataderos  2312  México D.F.   None   
3  Sales Representative                120 Hanover Sq.       London   None   
4   Order Administrator                Berguvsvägen  8        Luleå   None   

  PostalCode  Country           Phone             Fax  
0      12209  Germany     030-0074321     

Product table van AW in dataframe

In [12]:
query_product = "SELECT * FROM Production.Product"

# Execute the query and load data into a Pandas DataFrame
product_df = pd.read_sql(query_product, DB_AW_conn)

# Close the database connection

# Now you have the Product table in a Pandas DataFrame
product_df

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.00,...,None,None,None,NaN,NaN,2008-04-30,NaT,None,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,...,None,None,None,NaN,NaN,2008-04-30,NaT,None,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,...,None,None,None,NaN,NaN,2008-04-30,NaT,None,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0000,0.00,...,None,None,None,NaN,NaN,2008-04-30,NaT,None,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,...,None,None,None,NaN,NaN,2008-04-30,NaT,None,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.24,...,None,M,None,5.0,96.0,2013-05-30,NaT,None,71AB847F-D091-42D6-B735-7B0C2D82FC84,2014-02-08 10:01:36.827
500,996,HL Bottom Bracket,BB-9108,True,True,None,500,375,53.9416,121.49,...,None,H,None,5.0,97.0,2013-05-30,NaT,None,230C47C5-08B2-4CE3-B706-69C0BDD62965,2014-02-08 10:01:36.827
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,539.99,...,R,L,U,2.0,31.0,2013-05-30,NaT,None,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08 10:01:36.827
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,539.99,...,R,L,U,2.0,31.0,2013-05-30,NaT,None,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08 10:01:36.827


Product table van NW in dataframe

In [14]:
queryProductss = "SELECT * FROM Products"

# Execute the query and load data into a Pandas DataFrame
Productss_df = pd.read_sql(queryProductss, DB_NW_conn)

# Close the database connection

# Now you have the Customers table in a Pandas DataFrame
# print(Productss_df.head())  # Display the first few rows to verify
Productss_df

,ProductID,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,False
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,False
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,False
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,False
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,True
...,...,...,...,...,...,...,...,...,...,...
72,73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,False
73,74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,False
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,False
75,76,Lakkalikööri,23,1,500 ml,18.00,57,0,20,False


Merge Product

In [15]:

# Assuming you already have product_df and Products_df loaded from SQL

# Merge the two DataFrames on ProductID
merged_df = pd.merge(product_df, Productss_df, left_on='ProductID', right_on='ProductID', suffixes=('_AW', '_NW'))

# Display the merged DataFrame
# print(merged_df.head())

merged_df


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ModifiedDate,ProductName,SupplierID,CategoryID,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,0.0,...,2014-02-08 10:01:36.827,Chai,1,1,10 boxes x 20 bags,18.0,39,0,10,False
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,...,2014-02-08 10:01:36.827,Chang,1,1,24 - 12 oz bottles,19.0,17,40,25,False
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,...,2014-02-08 10:01:36.827,Aniseed Syrup,1,2,12 - 550 ml bottles,10.0,13,70,25,False
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,...,2014-02-08 10:01:36.827,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.0,53,0,0,False


CUSTOMER TABLE FROM AW IN DATAFRAME

SAles terrirtory mergerd with customer table 

In [23]:
queryCUSTOMERAW = "SELECT * FROM SALES.CUSTOMER"
queryTERITORY= "SELECT * FROM SALES.SalesTerritory"
Teritory_df= pd.read_sql(queryTERITORY,DB_AW_conn)
# Execute the query and load data into a Pandas DataFrame
CUSTOMERSAW_DF = pd.read_sql(queryCUSTOMERAW, DB_AW_conn)

# Close the database connection

# Now you have the Product table in a Pandas DataFrame
# CUSTOMERSAW_DF
# Teritory_df
merged_df = pd.merge(CUSTOMERSAW_DF, Teritory_df, on='TerritoryID', suffixes=('_Customer', '_Territory'))
merged_df


,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid_Customer,ModifiedDate_Customer,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid_Territory,ModifiedDate_Territory
0,1,NaN,934.0,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
2,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
3,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19815,30114,1985.0,1986.0,7,AW00030114,97154F3D-28F1-4B15-AE03-9518B781ECE3,2014-09-12 11:15:07.263,France,FR,Europe,4.772398e+06,2.396540e+06,0.0,0.0,BF806804-9B4C-4B07-9D19-706F2E689552,2008-04-30
19816,30115,1987.0,1988.0,6,AW00030115,E4CF8FD5-30A4-4B8E-8FD8-47032E255778,2014-09-12 11:15:07.263,Canada,CA,North America,6.771829e+06,5.693989e+06,0.0,0.0,06B4AF8A-1639-476E-9266-110461D66B00,2008-04-30
19817,30116,1989.0,1990.0,4,AW00030116,EC409609-D25D-41B8-9D15-A1AA6E89FC77,2014-09-12 11:15:07.263,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
19818,30117,1991.0,1992.0,4,AW00030117,6F08E2FB-1CD3-4F6E-A2E6-385669598B19,2014-09-12 11:15:07.263,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30


CUSTOMER TABLE FROM NW IN DATAFRAME

In [18]:
queryCUSTOMERSNW = "SELECT * FROM Customers"

# Execute the query and load data into a Pandas DataFrame
CustomersNW_df = pd.read_sql(queryCUSTOMERSNW, DB_NW_conn)

# Close the database connection

# Now you have the Customers table in a Pandas DataFrame
# print(Productss_df.head())  # Display the first few rows to verify
CustomersNW_df

,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,None,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,None,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,None,05023,Mexico,(5) 555-3932,None
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,None,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,None,S-958 22,Sweden,0921-12 34 65,0921-12 34 67
...,...,...,...,...,...,...,...,...,...,...,...
86,WARTH,Wartian Herkku,Pirkko Koskitalo,Accounting Manager,Torikatu 38,Oulu,None,90110,Finland,981-443655,981-443655
87,WELLI,Wellington Importadora,Paula Parente,Sales Manager,"Rua do Mercado, 12",Resende,SP,08737-363,Brazil,(14) 555-8122,None
88,WHITC,White Clover Markets,Karl Jablonski,Owner,305 - 14th Ave. S. Suite 3B,Seattle,WA,98128,USA,(206) 555-4112,(206) 555-4115
89,WILMK,Wilman Kala,Matti Karttunen,Owner/Marketing Assistant,Keskuskatu 45,Helsinki,None,21240,Finland,90-224 8858,90-224 8858


MERGE CUSTOMER TABLES From AW & NW

In [29]:
merged_df['CustomerID'] = merged_df['CustomerID'].astype(str)

# Convert CustomerID to string/object type in CustomersNW_df
CustomersNW_df['CustomerID'] = CustomersNW_df['CustomerID'].astype(str)
finalMergedCUstomers_df= pd.merge(merged_df, CustomersNW_df, on='CustomerID', how='left')

finalMergedCUstomers_df

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid_Customer,ModifiedDate_Customer,Name,CountryRegionCode,Group,...,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
0,1,NaN,934.0,1,AW00000001,3F5AE95E-B87D-4AED-95B4-C3797AFCB74F,2014-09-12 11:15:07.263,Northwest,US,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263,Northwest,US,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263,Southwest,US,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263,Southwest,US,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263,Southwest,US,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19815,30114,1985.0,1986.0,7,AW00030114,97154F3D-28F1-4B15-AE03-9518B781ECE3,2014-09-12 11:15:07.263,France,FR,Europe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19816,30115,1987.0,1988.0,6,AW00030115,E4CF8FD5-30A4-4B8E-8FD8-47032E255778,2014-09-12 11:15:07.263,Canada,CA,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19817,30116,1989.0,1990.0,4,AW00030116,EC409609-D25D-41B8-9D15-A1AA6E89FC77,2014-09-12 11:15:07.263,Southwest,US,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19818,30117,1991.0,1992.0,4,AW00030117,6F08E2FB-1CD3-4F6E-A2E6-385669598B19,2014-09-12 11:15:07.263,Southwest,US,North America,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


customer tabel in datawarehouse importeren

In [34]:


def insert_into_final_merged_customers(finalMergedCUstomers_df, export_conn):
    try:
        export_cursor = export_conn.cursor()

        # Define the CREATE TABLE query
        create_table_query = """
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='final_merged_customers' AND xtype='U')
        CREATE TABLE final_merged_customers (
            CustomerID VARCHAR(50),
            PersonID INT,
            StoreID INT,
            TerritoryID INT,
            AccountNumber VARCHAR(50),
            rowguid_Customer VARCHAR(50),
            ModifiedDate_Customer DATETIME,
            Name VARCHAR(255),
            CountryRegionCode VARCHAR(10),
            "Group" VARCHAR(50),
            SalesYTD FLOAT,
            SalesLastYear FLOAT,
            CostYTD FLOAT,
            CostLastYear FLOAT,
            rowguid_Territory VARCHAR(50),
            ModifiedDate_Territory DATETIME
        )
        """
        export_cursor.execute(create_table_query)
        
        # Iterate through each row in finalMergedCustomers_df and insert into the database
        for index, row in finalMergedCUstomers_df.iterrows():
            try:
                query = f"""
                INSERT INTO final_merged_customers (
                    CustomerID, PersonID, StoreID, TerritoryID, AccountNumber,
                    rowguid_Customer, ModifiedDate_Customer, Name, CountryRegionCode,
                    "Group", SalesYTD, SalesLastYear, CostYTD, CostLastYear,
                    rowguid_Territory, ModifiedDate_Territory
                ) 
                VALUES (
                    '{str(row['CustomerID']).replace("'", "''") if row['CustomerID'] is not None else 'NULL'}',
                    {row['PersonID'] if pd.notnull(row['PersonID']) else 'NULL'},
                    {row['StoreID'] if pd.notnull(row['StoreID']) else 'NULL'},
                    {row['TerritoryID'] if pd.notnull(row['TerritoryID']) else 'NULL'},
                    '{str(row['AccountNumber']).replace("'", "''") if row['AccountNumber'] is not None else 'NULL'}',
                    '{str(row['rowguid_Customer']).replace("'", "''") if row['rowguid_Customer'] is not None else 'NULL'}',
                    '{row['ModifiedDate_Customer'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(row['ModifiedDate_Customer']) else 'NULL'}',
                    '{str(row['Name']).replace("'", "''") if row['Name'] is not None else 'NULL'}',
                    '{str(row['CountryRegionCode']).replace("'", "''") if row['CountryRegionCode'] is not None else 'NULL'}',
                    '{str(row['Group']).replace("'", "''") if row['Group'] is not None else 'NULL'}',
                    {row['SalesYTD'] if pd.notnull(row['SalesYTD']) else 'NULL'},
                    {row['SalesLastYear'] if pd.notnull(row['SalesLastYear']) else 'NULL'},
                    {row['CostYTD'] if pd.notnull(row['CostYTD']) else 'NULL'},
                    {row['CostLastYear'] if pd.notnull(row['CostLastYear']) else 'NULL'},
                    '{str(row['rowguid_Territory']).replace("'", "''") if row['rowguid_Territory'] is not None else 'NULL'}',
                    '{row['ModifiedDate_Territory'].strftime('%Y-%m-%d %H:%M:%S') if pd.notnull(row['ModifiedDate_Territory']) else 'NULL'}'
                )
                """
                export_cursor.execute(query)
            except pyodbc.Error as e:
                print("Error:", e)
                print("Query:", query)

        export_conn.commit()
        print("Data successfully inserted into final_merged_customers table.")
    except pyodbc.Error as e:
        print("Error:", e)
    finally:
        export_cursor.close()

# Assuming finalMergedCustomers_df is already defined and export_conn is your pyodbc connection
insert_into_final_merged_customers(finalMergedCUstomers_df, export_conn)


Error: ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (0) (SQLExecDirectW)')


In [8]:

# Load data from the tables
newQuery = "SELECT * FROM [Sales].[Sales_SalesReason]"
sales_reason = pd.read_sql(newQuery, DB_AW_conn)

newQuery = "SELECT * FROM [Sales].[SalesOrderHeaderSalesReason]"
order_header_sales_reason = pd.read_sql(newQuery, DB_AW_conn)

newQuery = "SELECT * FROM [Sales].[SalesOrderHeader]"
order_header = pd.read_sql(newQuery, DB_AW_conn)

# Merge the DataFrames
merged_data = order_header.merge(order_header_sales_reason, on='SalesOrderID', how='left')
merged_data = merged_data.merge(sales_reason, on='SalesReasonID', how='left')

# Fill NaN values with appropriate defaults
merged_data['Name'].fillna('', inplace=True)
merged_data['ReasonType'].fillna('', inplace=True)

# Insert the merged data into a new table
export_cursor = export_conn.cursor()

for index, row in merged_data.iterrows():
    try:
        query = f"""
        INSERT INTO MergedSalesData (
            SalesOrderID, RevisionNumber, OrderDate, DueDate, ShipDate, Status,
            SalesOrderNumber, PurchaseOrderNumber, AccountNumber, CustomerID,
            SalesPersonID, TerritoryID, BillToAddressID, ShipToAddressID,
            ShipMethodID, CreditCardID, SubTotal, TaxAmt, TotalDue,
            SalesReasonID, Name, ReasonType
        ) VALUES (
            {row['SalesOrderID']}, {row['RevisionNumber']}, '{row['OrderDate']}', '{row['DueDate']}', 
            '{row['ShipDate']}', {row['Status']}, '{row['SalesOrderNumber']}', 
            '{row['PurchaseOrderNumber']}', '{row['AccountNumber']}', {row['CustomerID']}, 
            {row['SalesPersonID']}, {row['TerritoryID']}, {row['BillToAddressID']}, 
            {row['ShipToAddressID']}, {row['ShipMethodID']}, {row['CreditCardID']}, 
            {row['SubTotal']}, {row['TaxAmt']}, {row['TotalDue']}, {row['SalesReasonID']}, 
            '{row['Name']}', '{row['ReasonType']}'
        )
        """
        export_cursor.execute(query)
    except pyodbc.Error as e:
        print("An error occurred:", e)
        print(query)

export_conn.commit()
export_cursor.close()


DatabaseError: Execution failed on sql 'SELECT * FROM [Sales].[Sales_SalesReason]': ('42S02', "[42S02] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid object name 'Sales.Sales_SalesReason'. (208) (SQLExecDirectW)")